In [9]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [10]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [11]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [12]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [13]:
df1['target']=df1['target'].map({1:0,0:1})

In [14]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_1'],axis=1)

In [15]:
df1.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,118300,123472,117906,117908,0,117961118300
1,17183,1540,118343,123125,118536,118539,0,117961118343
2,36724,14457,118220,117884,267952,117880,0,118219118220
3,36135,5396,118343,119993,240983,118322,0,117961118343
4,42680,5905,117930,119569,123932,119325,0,117929117930


# Combinations with brute force 


In [16]:
from itertools import combinations

cols=list(df1.columns)
cols.remove('target')
d=df1.copy()

for j in combinations(cols,2):
    d[j[0]+'_'+j[1]]=df1[j[0]]+df1[j[1]]
    
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 29 columns):
RESOURCE                          32769 non-null object
MGR_ID                            32769 non-null object
ROLE_ROLLUP_2                     32769 non-null object
ROLE_DEPTNAME                     32769 non-null object
ROLE_FAMILY_DESC                  32769 non-null object
ROLE_CODE                         32769 non-null object
target                            32769 non-null int64
ROLLUP_12                         32769 non-null object
RESOURCE_MGR_ID                   32769 non-null object
RESOURCE_ROLE_ROLLUP_2            32769 non-null object
RESOURCE_ROLE_DEPTNAME            32769 non-null object
RESOURCE_ROLE_FAMILY_DESC         32769 non-null object
RESOURCE_ROLE_CODE                32769 non-null object
RESOURCE_ROLLUP_12                32769 non-null object
MGR_ID_ROLE_ROLLUP_2              32769 non-null object
MGR_ID_ROLE_DEPTNAME              32769 non-null obj

In [17]:
def AST_ecoding(train,test,columns,target,weight):
    t=train.copy()
    v=test.copy()
    gm=t[target].mean()
    for col in columns:
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        t[col+'_enc']=t[col].map(gb['smooth'])
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
    v=v.drop(columns,axis=1)
    t=t.drop(columns,axis=1)
    return t,v

In [18]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    return auc
    #print('AUC_ROC= ',auc)
    #print('Accuracy= ',acc)
    #print('confusion matrix:\n',confusion_matrix(y,pred))

In [19]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
lgb=LGBMClassifier(n_estimators=200)
lr=LogisticRegression()

In [20]:
train,test=train_test_split(d,test_size=0.3,random_state=123)

columns=list(d.columns)
columns.remove('target')
target='target'
t,v=AST_ecoding(train,test,columns,target,20)

In [21]:
t.shape

(22938, 29)

In [22]:
t=t.sample(n=10000)
t[target].value_counts()

0    9424
1     576
Name: target, dtype: int64

In [23]:

xtr=t.drop('target',axis=1)
xts=v.drop(target,axis=1)
ytr=t[target]
yts=v[target]
cols=list(xtr.columns)


In [ ]:
high=0
high2=0
for i in tqdm(range(1,xtr.shape[1])):
    for j in combinations(cols,i):
        mod=lr.fit(xtr[list(j)],ytr)
        auc=aucroc(mod,xts[list(j)],yts)
        if auc>high:
            high=auc
            best_feat=list(j)
        elif auc>high2:
            high2=auc
            best_feat_2=list(j)
print('Best Score=',high)
print('best_features=',best_feat)
print('\n2nd Best Score=',high2)
print('best_features_2=',best_feat_2)
            
            
        
    


 22%|██▏       | 6/27 [6:35:05<37:06:11, 6360.56s/it]